## Evaluation of translators to deterministic automata with additional optimizations

This notebook contains evaluation of `ltl3tela -D1`, `ltl2tgba -DG`, Delag and Rabinizer 4 where we applied the Spot postprocessing procedures (using the `autfilt` tool) to output of Delag and Rabinizer 4.

In [38]:
from ltlcross_runner import LtlcrossRunner
from IPython.display import display
import pandas as pd
import spot
import sys
import re
spot.setup(show_default='.a')
pd.options.display.float_format = '{: .0f}'.format
pd.options.display.latex.multicolumn_format = 'c'
from tables_utils import split_cols, high_min, high_max, highlight_by_level
from tables_utils import fix_latex, fix_type, fix_tool
from tables_utils import cummulative_to_latex

In [39]:
rerun = False

In [56]:
%%bash
ltl3tela -v
ltl2tgba --version
delag --version
ltl2dgra --version # Rabinizer 4
autfilt --version

LTL3TELA 2.0.0 (using Spot 2.7.4)
ltl2tgba (spot) 2.7.4

Copyright (C) 2019  Laboratoire de Recherche et Développement de l'Epita.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.
Name: owl
Version: 18.06
Name: owl
Version: 18.06
autfilt (spot) 2.7.4

Copyright (C) 2019  Laboratoire de Recherche et Développement de l'Epita.
License GPLv3+: GNU GPL version 3 or later <http://gnu.org/licenses/gpl.html>.
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.


In [41]:
def generate(n=1000,func=(lambda x: True),filename=None,priorities='',ap=['a','b','c','d','e']):
    if filename is None:
        file_h = sys.stdout
    else:
        file_h = open(filename,'w')
    f = spot.randltl(ap,
                     ltl_priorities=priorities,
                     simplify=3,tree_size=15).relabel_bse(spot.Abc)
    i = 0
    printed = set()
    while(i < n):
        form = next(f)
        if form in printed:
            continue
        if func(form) and not form.is_tt() and not form.is_ff():
            print(form,file=file_h)
            printed.add(form)
            i += 1

In [42]:
f_rand = 'formulae/atva19/rand.ltl'
f_patterns = 'formulae/atva19/patterns.ltl'
# generate(1000, filename = f_rand)

### Deterministic automata

In [43]:
d_tools = {
    "ltl3tela-D1": "ltl3tela -D1 -f %f > %O",
    "ltl2tgba-DG": "ltl2tgba -DG %f > %O",
    "delag": "delag %f | autfilt -G --deterministic --high > %O",
    "rabinizer4": "ltl2dgra %f | autfilt -G --deterministic --high > %O"
}
d_order = ["ltl3tela-D1", "ltl2tgba-DG", "delag", "rabinizer4"]
d_cols = ["states", "edges", "acc"]

In [44]:
d_csv_rand = 'formulae/atva19/det.rand.autfilt.csv'
d_data_rand = LtlcrossRunner(d_tools, formula_files = [f_rand], res_filename = d_csv_rand, cols = d_cols)
if rerun:
    d_data_rand.run_ltlcross(automata = False, timeout = '60')
d_data_rand.parse_results()

In [45]:
det_rand = d_data_rand.cummulative(col = d_cols).unstack(level = 0).loc[d_order, d_cols]
det_rand

column,states,edges,acc
tool,,,
ltl3tela-D1,5934,18520,1268
ltl2tgba-DG,6799,24131,1575
delag,6468,24168,2203
rabinizer4,6450,24454,1915


In [46]:
d_csv_patterns = 'formulae/atva19/det.patterns.autfilt.csv'
d_data_patterns = LtlcrossRunner(d_tools, formula_files = [f_patterns], res_filename = d_csv_patterns, cols = d_cols)
if rerun:
    d_data_patterns.run_ltlcross(automata = False, timeout = '60')
d_data_patterns.parse_results()

In [47]:
det_to = pd.DataFrame(d_data_patterns.get_error_count(),columns=['TO.literature'])
det_err = pd.DataFrame(d_data_patterns.get_error_count('parse error',False),columns=['PE.literature'])
det_lit = d_data_patterns.cummulative(col = d_cols).unstack(level = 0).loc[d_order, d_cols]
det_lit = pd.concat([det_lit,det_to,det_err],axis=1,join='inner',sort=False)
det_lit

,states,edges,acc,TO.literature,PE.literature
tool,,,,,
ltl3tela-D1,2536,10641,454,39,0
ltl2tgba-DG,3905,26643,652,20,0
delag,2732,18142,774,11,0
rabinizer4,2766,11091,749,12,0


In [48]:
to = d_data_rand.exit_status
to[to != "ok"].dropna(how='all')

,tool,delag,ltl2tgba-DG,ltl3tela-D1,rabinizer4
form_id,formula,,,,


In [49]:
det_tmp = pd.merge(det_rand, det_lit, suffixes=('.random','.literature'),on='tool')
det_tmp

,states.random,edges.random,acc.random,states.literature,edges.literature,acc.literature,TO.literature,PE.literature
tool,,,,,,,,
ltl3tela-D1,5934,18520,1268,2536,10641,454,39,0
ltl2tgba-DG,6799,24131,1575,3905,26643,652,20,0
delag,6468,24168,2203,2732,18142,774,11,0
rabinizer4,6450,24454,1915,2766,11091,749,12,0


In [50]:
det = split_cols(det_tmp,'.').swaplevel(axis=1)
det

random              literature                   
            states  edges   acc     states  edges  acc  TO PE
tool                                                         
ltl3tela-D1   5934  18520  1268       2536  10641  454  39  0
ltl2tgba-DG   6799  24131  1575       3905  26643  652  20  0
delag         6468  24168  2203       2732  18142  774  11  0
rabinizer4    6450  24454  1915       2766  11091  749  12  0

In [51]:
rerun = False

### Formulae excluded from the evaluation

In [52]:
d_fails = d_data_patterns.values[d_data_patterns.values.isnull().any(axis = 1)]['states']\
    .join(d_data_patterns.exit_status, lsuffix = '.states', rsuffix = '.response')
for tool in d_order:
    d_fails[tool] = d_fails[tool + '.states'].combine_first(d_fails[tool + '.response'])
d_fails_out = d_fails[d_order]
d_fails_out

,tool,ltl3tela-D1,ltl2tgba-DG,delag,rabinizer4
form_id,formula,,,,
103,GFz <-> (GFa1 & GFa2 & GFa3 & GFa4),1,1,1,exit code
104,GFz <-> (GFa1 & GFa2 & GFa3 & GFa4 & GFa5),timeout,1,1,exit code
129,(FGp2 | GFp1) & (FGp3 | GFp2) & (FGp4 | GFp3) & (FGp5 | GFp4) & (GFp5 | FGp6),1,1,1,exit code
181,G(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X(!p0 | X!p0))))))))))),timeout,12,12,12
193,c & X(a1 | b1 | d) & G(c -> !d) & G((a1 | b1) -> X(a2 | b2)) & G((a2 | b2) -> X(c & X(a1 | b1 | d | Gc))) & (!d U (d & X((a1 | b1) & XXGc))) & F(c & X(!c & (((a1 & F(d & Fa1)) | (b1 & F(d & Fb1)) | (a2 & F(d & Fa2)) | (b2 & F(d & Fb2))) U c))) & G((c | d) -> !(a1 | a2 | b1 | b2)) & G((a1 -> !b1) & (a2 -> !b2)),timeout,timeout,82,82
194,c & X(a1 | b1 | d) & G(c -> !d) & G(((a1 | b1) -> X(a2 | b2)) & ((a2 | b2) -> X(a3 | b3))) & G((a3 | b3) -> X(c & X(a1 | b1 | d | Gc))) & (!d U (d & X((a1 | b1) & XXXGc))) & F(c & X(!c & (((a1 & F(d & Fa1)) | (b1 & F(d & Fb1)) | (a2 & F(d & Fa2)) | (b2 & F(d & Fb2)) | (a3 & F(d & Fa3)) | (b3 & F(d & Fb3))) U c))) & G((c | d) -> !(a1 | a2 | a3 | b1 | b2 | b3)) & G((a1 -> !b1) & (a2 -> !b2) & (a3 -> !b3)),timeout,timeout,2240,timeout
195,c & X(a1 | b1 | d) & G(c -> !d) & G(((a1 | b1) -> X(a2 | b2)) & ((a2 | b2) -> X(a3 | b3)) & ((a3 | b3) -> X(a4 | b4))) & G((a4 | b4) -> X(c & X(a1 | b1 | d | Gc))) & (!d U (d & X((a1 | b1) & XXXXGc))) & F(c & X(!c & (((a1 & F(d & Fa1)) | (b1 & F(d & Fb1)) | (a2 & F(d & Fa2)) | (b2 & F(d & Fb2)) | (a3 & F(d & Fa3)) | (b3 & F(d & Fb3)) | (a4 & F(d & Fa4)) | (b4 & F(d & Fb4))) U c))) & G((c | d) -> !(a1 | a2 | a3 | a4 | b1 | b2 | b3 | b4)) & G((a1 -> !b1) & (a2 -> !b2) & (a3 -> !b3) & (a4 -> !b4)),timeout,timeout,timeout,timeout
196,c & X(a1 | b1 | d) & G(c -> !d) & G(((a1 | b1) -> X(a2 | b2)) & ((a2 | b2) -> X(a3 | b3)) & ((a3 | b3) -> X(a4 | b4)) & ((a4 | b4) -> X(a5 | b5))) & G((a5 | b5) -> X(c & X(a1 | b1 | d | Gc))) & (!d U (d & X((a1 | b1) & XXXXXGc))) & F(c & X(!c & (((a1 & F(d & Fa1)) | (b1 & F(d & Fb1)) | (a2 & F(d & Fa2)) | (b2 & F(d & Fb2)) | (a3 & F(d & Fa3)) | (b3 & F(d & Fb3)) | (a4 & F(d & Fa4)) | (b4 & F(d & Fb4)) | (a5 & F(d & Fa5)) | (b5 & F(d & Fb5))) U c))) & G((c | d) -> !(a1 | a2 | a3 | a4 | a5 | b1 | b2 | b3 | b4 | b5)) & G((a1 -> !b1) & (a2 -> !b2) & (a3 -> !b3) & (a4 -> !b4) & (a5 -> !b5)),timeout,timeout,timeout,timeout
198,c & X(d | y) & G(y -> X((a | b) & Xz)) & G(z -> X((a | b) & X(c & X(d | y | Gc)))) & (!d U (d & X(y & XXXXGc))) & F(c & X(!c & (((y -> X((a & F(d & F(y & Xa))) | (b & F(d & F(y & Xb))))) & (z -> X((a & F(d & F(z & Xa))) | (b & F(d & F(z & Xb)))))) U c))) & G(!(a & b) & !(a & c) & !(a & d) & !(a & y) & !(b & c) & !(b & d) & !(b & y) & !(c & d) & !(c & y) & !(d & y) & !(a & z) & !(b & z) & !(c & z) & !(d & z) & !(y & z)),timeout,timeout,147,timeout


In [53]:
def large_diff(res, tools, threshold):
    df = res.values.dropna()['states']
    df['diff'] = df.loc[:, tools].max(axis = 1) - df.loc[:, tools].min(axis = 1)
    return df[df['diff'] > threshold][tools]

In [54]:
large_diff(d_data_patterns, d_tools, 20)

,tool,ltl2tgba-DG,ltl3tela-D1,delag,rabinizer4
form_id,formula,,,,
72,G(p0 -> (((p1 & X(!p2 U p3)) -> X(!p2 U (p3 & Fp4))) U (p2 | G((p1 & X(!p2 U p3)) -> X(!p2 U (p3 & Fp4)))))),16,6,62,34
77,G(p0 -> ((p1 -> (!p2 U (!p2 & p3 & X(!p2 U p4)))) U (p2 | G(p1 -> (p3 & XFp4))))),11,10,43,11
82,G(p0 -> ((p1 -> (!p2 U (!p2 & p3 & !p4 & X((!p2 & !p4) U p5)))) U (p2 | G(p1 -> (p3 & !p4 & X(!p4 U p5)))))),11,11,82,11
108,GF(a <-> XXXXa),16,16,16,65
109,GF(a <-> XXXXXa),32,32,32,161
172,G((!(p1 <-> Xp1) | !(p0 <-> Xp0) | !(p2 <-> Xp2) | !(p3 <-> Xp3)) -> (X!p4 & X(!(!(p1 <-> Xp1) | !(p0 <-> Xp0) | !(p2 <-> Xp2) | !(p3 <-> Xp3)) U p4))),81,34,33,33
235,FG(a | b) | FG(!a | Xb) | FG(a | XXb) | FG(!a | XXXb) | FG(a | XXXXb),16,16,16,83
337,GFa2 U G(GFa1 U G(GFa0 U Xb)),257,11,3,3
338,GFa2 U G(GFa1 U G(GFa0 U XXb)),258,12,4,4


In [55]:
large_diff(d_data_rand, d_tools, 10)

,tool,ltl2tgba-DG,ltl3tela-D1,delag,rabinizer4
form_id,formula,,,,
23,Fa U (F(!b | (c & XXb) | (!c & XX!b)) R d),16,7,21,8
46,((a M b) U Xc) W Gd,62,12,18,13
57,GF(a | (b R XFc)),15,2,1,1
58,(a & G((XFb & (c U Xd)) | (XG!b & (!c R X!d)))) | (!a & F((XFb & (!c R X!d)) | ((c U Xd) & XG!b))),20,17,28,21
84,G(G(a & (b | (c M Xd))) | (F!a & F(!b & (!c W X!d)))),32,11,12,18
118,XF(((a M Gb) & ((XFc & (d | Fe)) | (!d & XG!c & G!e))) | ((!a W F!b) & ((!d & XFc & G!e) | ((d | Fe) & XG!c)))),116,79,125,125
143,F((a & X((!b | XG!c) U !c)) | (!a & X((b & XFc) R c))),13,6,21,8
195,(!a | GFb) U (Gb | (a W Gc)),24,14,7,7
215,G(Ga | Xb | F(c & Gd)),30,16,18,22
